In [1]:
import os
import time

import numpy as np
import torch
import argparse

from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer, 
                          Phi3ForCausalLM,
                          Phi3Config,
                          pipeline,
                          )
from transformers.pipelines.pt_utils import KeyDataset
from huggingface_hub import hf_hub_download
import torch_tensorrt

/home/elicer/miniconda3/envs/yaikids/lib/python3.10/site-packages/modelopt/torch/quantization/tensor_quant.py:92: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  scaled_e4m3_abstract = torch.library.impl_abstract("trt::quantize_fp8")(
/home/elicer/miniconda3/envs/yaikids/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")


In [2]:
parser = argparse.ArgumentParser(description="TensorRT")
parser.add_argument("--model_dir", type=str, default="models/")
parser.add_argument("--model_name", type=str, default="Phi-3-medium-4k-instruct")
parser.add_argument("--data_dir", type=str, default="data/")
parser.add_argument("--data_name", type=str, default="test_dataset.jsonl")
parser.add_argument('--seed',type=int, default=0)
parser.add_argument("--batch_size", type=int, default=1)
parser.add_argument('--dtype',type=str, default="auto")
parser.add_argument('--n', type=int, default=1)
parser.add_argument('--temperature', type=float, default=0.0)

config = parser.parse_args([])

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model_id = os.path.join(config.model_dir, config.model_name)

# with torch.no_grad():
model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="cuda",
        torch_dtype=torch.half,
        trust_remote_code=True,
        torchscript=True,
        attn_implementation="eager",
        use_cache=False,
    )
    
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
dummy_message = "Can you provide ways to eat combinations of bananas and dragonfruits?"

model_inputs = tokenizer(dummy_message, return_tensors="pt")
input_ids = model_inputs.input_ids.to(DEVICE)

jit_model = torch.jit.trace(model, input_ids)

# def export_llm(model, inputs, min_seq_len=1, max_seq_len=64):
#     seq_len = torch.export.Dim("seq_len", min=min_seq_len, max=max_seq_len)
    
#     with torch.no_grad():
#         ep = torch.export.export(
#             model, (inputs,), dynamic_shapes=({1: seq_len},), strict=False
#         )
#     return ep 

# model_ep = export_llm(model, input_ids)

  warnings.warn(



  assert padding_idx < weight.size(0), "Padding_idx must be within num_embeddings"

  if input_shape[-1] > 1 or self.sliding_window is not None:

  if attn_weights.size() != (bsz, self.num_heads, q_len, kv_seq_len):



In [ ]:
# dummy_messages = [
#     {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"},
#     {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."},
#     {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
# ]

# # input_ids = tokenizer.apply_chat_template(dummy_messages, 
# #                                         add_generation_prompt=True, 
# #                                         tokenize=True,
# #                                         return_tensors="pt",
# #                                         padding=True).to(DEVICE)

# dummy_message = "Can you provide ways to eat combinations of bananas and dragonfruits?"
# model_inputs = tokenizer(dummy_message, return_tensors="pt", padding=True)
# input_ids = model_inputs.input_ids
# traced_model = torch.jit.trace(model.forward, input_ids)

In [37]:
# [
#     torch_tensorrt.Input(shape=[1, 16], dtype=torch.half),
#     torch_tensorrt.Input(shape=[4, 16], dtype=torch.half),
#     torch_tensorrt.Input(shape=[16, 16], dtype=torch.half),
# ]

Input(shape=(1, 16), dtype=dtype.f16, format=memory_format.linear, domain=[0.0, 2.0))

In [ ]:
torch.export.export(model, (input_ids, ))

In [10]:
trt_model = torch_tensorrt.dynamo.compile(
    jit_model,
    inputs=input_ids,
    enabled_precisions={torch.half},
    min_block_size=1,
    truncate_long_and_double=True,
    disable_t32=True
)

system_message = "You are a helpful AI Assistant. Help users by replying to their queries and make sure the responses are polite. Do not hallucinate."
PROMPT = f"<|assistant|>\n{system_message}<|end|>"

  trt_model = torch_tensorrt.dynamo.compile(



AssertionError: Input graph should be an ExportedProgram but got type <class 'torch.jit._trace.TopLevelTracedModule'>

In [ ]:
torch.jit.save(trt_model, 'trt.ts')

In [ ]:
dummy_message = "Can you provide ways to eat combinations of bananas and dragonfruits?"


input_ids = tokenizer(dummy_message, return_tensors="pt").to(DEVICE)

traced_model = torch.jit.trace(model, (input_ids["input_ids"], input_ids["attention_mask"]))

In [4]:
torch.cuda.synchronize()
start = time.perf_counter()

data = load_dataset("json", data_files="./data/test_dataset.jsonl")['train']
messages = data['message']
messages.insert(0, PROMPT)

token_ids = tokenizer.apply_chat_template(messages, 
                                        add_generation_prompt=True, 
                                        tokenize=False,)

toekn_ids.insert(0, PROMPT)

inputs = tokenizer(toekn_ids, return_tensors="pt", padding=True)
inputs = {k: v.type(torch.int32).to(DEVICE) for k, v in inputs.items()}

with torch.inference_mode():
    outs = trt_model.generate(
        input_ids=inputs.input_ids,
        **generation_args
    )

generated_texts = tokenizer.batch_decode(outs, skip_special_tokens=True, clean_up_tokenization_spaces=False)
results = [
    [{"generated_text": text.rpartition('\n')[2]}]
    for text in generated_texts[1:]
]

torch.cuda.synchronize()
end = time.perf_counter()

NameError: name 'torch' is not defined

In [ ]:
print("===== Answers =====")
correct = 0
for i, out in enumerate(results):
    correct_answer = data[i]["answer"]
    answer = out[0]["generated_text"].lstrip().replace("\n","")
    if answer == correct_answer:
        correct += 1
    print(answer)
 
print("===== Perf result =====")
print("Elapsed_time: ", end-start)
print(f"Correctness: {correct}/{len(data)}")